In [1]:
INPUT_FILE_NAME = 'log_sample.csv'
#'small_log.csv'
#'log_sample.csv'
LABEL_TIME_PART = 0.2

In [2]:
import csv
import numpy as np
from tqdm import notebook

In [3]:
class Event:
    def __init__(self, timestamp, group_id, device_id, event_id, event_count):
        self.timestamp = int(timestamp)
        self.group_id = group_id
        self.event_id = event_id
        self.device_id = device_id
        if event_count:
            self.count = int(event_count)
        else:
            self.count = 0

In [62]:
def read_from_file_to_event_list():
    events = []
    with open(INPUT_FILE_NAME, 'r') as fin:
        for row in notebook.tqdm(csv.reader(fin, delimiter=',')):
            if (row[12].split('.')[0]) and row[5] == "productivity":
                events.append(Event(row[3], row[5], row[7], row[10], row[12].split('.')[0]))
    return events        
            

In [63]:
events = read_from_file_to_event_list()

In [64]:
def get_user_to_time_period(events):
    user_to_min_time = {}
    user_to_max_time = {}
    labels = {}

    for event in notebook.tqdm(events):
        if event.device_id not in user_to_min_time.keys():
            user_to_min_time[event.device_id] = event.timestamp
        else:
            user_to_min_time[event.device_id] = min(user_to_min_time[event.device_id], event.timestamp)
        if event.device_id not in user_to_max_time.keys():
            user_to_max_time[event.device_id] = event.timestamp
        else:
            user_to_max_time[event.device_id] = max(user_to_max_time[event.device_id], event.timestamp)

    user_to_time_period = {}
    
    for user in user_to_min_time.keys():
        min_time = user_to_min_time[user]
        max_time = user_to_max_time[user]
        user_to_time_period[user] = int((max_time - min_time) / 1000 /60/60/24)
    return user_to_time_period  
        
    

In [65]:
user_to_time_period = get_user_to_time_period(events).values()

In [66]:
print(len(user_to_time_period))

686


In [67]:
period_to_count = np.zeros(32)
for period in user_to_time_period:
    period_to_count[period] += 1


In [68]:
for i in range(len(period_to_count)):
    print(str(i) + " days: " + str(int(period_to_count[i])) + " users")

0 days: 192 users
1 days: 54 users
2 days: 32 users
3 days: 23 users
4 days: 25 users
5 days: 26 users
6 days: 16 users
7 days: 20 users
8 days: 17 users
9 days: 13 users
10 days: 11 users
11 days: 14 users
12 days: 12 users
13 days: 11 users
14 days: 9 users
15 days: 10 users
16 days: 9 users
17 days: 12 users
18 days: 10 users
19 days: 12 users
20 days: 7 users
21 days: 10 users
22 days: 10 users
23 days: 11 users
24 days: 13 users
25 days: 10 users
26 days: 8 users
27 days: 5 users
28 days: 16 users
29 days: 31 users
30 days: 37 users
31 days: 0 users


In [69]:
def get_user_to_time_periods(events):
    user_to_timestamps = {}

    for event in notebook.tqdm(events):
        if event.device_id in user_to_timestamps.keys():
            user_to_timestamps[event.device_id].append(event.timestamp)
        else:
            user_to_timestamps[event.device_id] = [event.timestamp]
      
    user_to_time_periods = {}
    for user in notebook.tqdm(user_to_timestamps.keys()):
        user_to_timestamps[user] = sorted(user_to_timestamps[user])
        user_to_time_periods[user] = []
        for i in range(len(user_to_timestamps[user]) - 1):
            diff = int((user_to_timestamps[user][i + 1] - user_to_timestamps[user][i]) / 1000 / 60)
            if diff > 0:
                user_to_time_periods[user].append(diff)
    return user_to_time_periods
    

In [70]:
user_to_time_periods = get_user_to_time_periods(events)

In [79]:
activity = []
for value in user_to_time_periods.values():
    cnt = 0
    for elem in value:
        if elem <= 60:
            cnt += elem
        else:
            #print("active(" + str(cnt), end=') ')
            activity.append(cnt)
            cnt = 0
            #print("stopped(" + str(elem), end=') ')
            #print()
    #print("active(" + str(cnt), end=') ')
    activity.append(cnt)  
#print(activity)
x = np.percentile(activity, 95)
print(x)

x1 = np.mean(activity)
print(x1)

x2 = np.median(activity)
print(x2)
    #print("END")

158.89999999999964
36.329071419346505
7.0


In [80]:
x / 60

2.648333333333327

In [81]:
sorted(activity, reverse=True)

[541,
 530,
 517,
 500,
 488,
 481,
 470,
 464,
 455,
 446,
 440,
 435,
 430,
 424,
 404,
 397,
 390,
 385,
 382,
 382,
 370,
 366,
 363,
 359,
 353,
 349,
 349,
 345,
 334,
 334,
 331,
 331,
 330,
 329,
 329,
 329,
 325,
 325,
 325,
 322,
 322,
 321,
 320,
 309,
 303,
 291,
 291,
 291,
 290,
 288,
 287,
 285,
 285,
 283,
 282,
 281,
 281,
 279,
 277,
 277,
 277,
 276,
 276,
 273,
 273,
 273,
 272,
 272,
 272,
 271,
 267,
 266,
 264,
 263,
 261,
 260,
 258,
 258,
 258,
 258,
 257,
 257,
 256,
 255,
 254,
 254,
 253,
 253,
 252,
 251,
 249,
 248,
 247,
 247,
 246,
 244,
 243,
 241,
 241,
 240,
 240,
 239,
 238,
 238,
 238,
 236,
 235,
 234,
 233,
 233,
 232,
 232,
 232,
 231,
 231,
 231,
 230,
 228,
 228,
 228,
 228,
 227,
 227,
 226,
 225,
 225,
 225,
 225,
 224,
 224,
 223,
 223,
 223,
 223,
 222,
 222,
 221,
 221,
 220,
 220,
 220,
 219,
 218,
 216,
 216,
 215,
 215,
 215,
 214,
 214,
 214,
 213,
 213,
 213,
 213,
 213,
 212,
 211,
 211,
 210,
 210,
 209,
 209,
 209,
 209,
 209,
 209

In [82]:
541 / 60

9.016666666666667